In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from ramannoodles import spectrafit
from ramannoodles import shoyu
import pandas as pd
import math

In [4]:
#If you re-call the method, it will tell you that the files are already downloaded.
shoyu.initialize_standard_library()
# open spectra library
shoyu_data_dict = pickle.load(open('../raman_spectra/shoyu_data_dict.p', 'rb'))
# list keys
sorted(shoyu_data_dict.keys())

file already in raman_spectra folder
WATER loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
CARBON MONOXIDE loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
CARBON DIOXIDE loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
FORMIC ACID loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
2-PROPANOL loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
ETHYL ALCOHOL loaded into the dictionary - shoyu_data_dict.p
file already in raman_spectra folder
DIMETHYL KETONE loaded into the dictionary - shoyu_data_dict.p


['2-PROPANOL',
 'CARBON DIOXIDE',
 'CARBON MONOXIDE',
 'DIMETHYL KETONE',
 'ETHYL ALCOHOL',
 'FORMIC ACID',
 'WATER']

In [ ]:
compound_1 = shoyu_data_dict['WATER']
compound_2 = shoyu_data_dict['CARBON MONOXIDE']
spectra_x, spectra_y = shoyu.combine_spectra(compound_1, compound_2, plot = True)

In [ ]:
# Exploring a basic example from lmfit documentation
import lmfit
import numpy as np
x = np.linspace(0.3, 10, 100)
np.random.seed(0)
y = 1/(0.1*x) + 2 + 0.1*np.random.randn(x.size)
pars = lmfit.Parameters()
pars.add_many(('a', 0.1), ('b', 1))
def residual(p):
    return 1/(p['a']*x) + p['b'] - y

In [ ]:
# Running a fit 
mini = lmfit.Minimizer(residual,pars)
result = mini.minimize()
print(mini.leastsq())
print(lmfit.fit_report(result.params))

In [ ]:
# Getting a basic confidence interval
ci = lmfit.conf_interval(mini,result)
lmfit.printfuncs.report_ci(ci)

In [ ]:
# using quantiles got the sigmas and using the trace

In [ ]:
#Bootstrapping is also an option

In [ ]:
def peak_1D_score(rowA,rowB,scoremax):
    """
    Returns scores with respect to the repricoal of the 
    calculated Euclidean distance between peaks
    #√((x1-x2)^2) in 1D
    #√((x1-x2)^2 + (y1-y2)^2) in 2D

    Parameters:
        row A (list):  input list
        row B (list): input list
        scoremax (float): Euclidean reciprocal score divided by max score

    Returns:
        scores (list): Euclidean reciprocal scores
        peaks (tuple): peaks associated with scores
    """
    scores = []
    peaks=[]
    

    for i in range(len(rowA)):
        for j in range(len(rowB)):
            distance = np.where((rowA[i] - rowB[j]>50),np.nan,math.sqrt(sum([math.pow(rowA[i] - rowB[j], 2)])))
            if (1/(distance + 1)>.02): # Score for peaks less than 50 units apart
                scores.append((((1/(distance + 1))/scoremax)))
                peaks.append((rowA[i],rowB[j]))
            else:
                pass
    return scores,peaks
#function calculates distance start

def score_max(list_input, row,k):
    """
    Returns list of scores with respect to its output max score

    Parameters:
        list_input (list):  input list
        row (list): input list
        k (int): input integer used to sort the scores / kth highest score

    Returns:
        maxscores (list): Euclidean reciprocal score divided by max score
        maxpeaks (tuple): peaks associated with max scores
    """
    try:
        maxscores,maxpeaks = peak_1D_score(list_input,row,sorted(set(peak_1D_score(list_input,row,1)[0][:]))[-k])
    
    except Exception as e:
        
        maxscores,maxpeaks = peak_1D_score(list_input,row, scoremax=1)
        
    return maxscores,maxpeaks
def score_sort(list_input, row,k):
    """
    Returns list of scores sorted

    Parameters:
        list_input (list):  input list
        row (list): input list
        k (int): input integer used to sort the scores / kth highest score

    Returns:
        sortedscores (list): sorted Euclidean distances
    """
    sortedscores = []
    sortedscores.append(score_max(list_input,row,k))
    sortedscores.sort()
    return sortedscores

In [ ]:
def test_peak_1D_score():
    """Evaluates the functionality of the peak_1D_score function"""
    # Initialize the test arguments 
    row_i=[0,1]
    row_j=[2,1]
    # Run Function
    testscore=peak_1D_score(row_i,row_j,1)[0][:]
    testpeaks=peak_1D_score(row_i,row_j,1)[1][:]
    # make assertions
    assert len(row_i) == len(row_j), 'Input lengths do not match'
    assert testscore <= .02, 'Output value outside acceptable range'
    assert isinstance(testscore, np.int), 'Output is not a numpy.float'

def test_score_max():
    """Evaluates the functionality of the score_max function"""
    # Initialize the test arguments 
    maxscores,maxpeaks = peak_1D_score(list_input,row,sorted(set(peak_1D_score(list_input,row,1)[0][:]))[-k])
    # Run Function
    assert len(d_train.index) == len(sorteddistances), 'Output array length different than DataFrame length'
    # make assertions
    
    return maxscores,maxpeaks
def test_score_sort(list_input, row,k):
    """Evaluates the functionality of the score_sort function"""
    # Initialize the test arguments 
    sortedscores = []
    # Run Function
    sortedscores=score_sort(list_input, row,k)
    # make assertions
    sortedscores.sort()
    return sortedscores

In [ ]:
compound_1 = shoyu_data_dict['WATER']
x_water = compound_1['x']
y_water = compound_1['y']
compound_2 = shoyu_data_dict['CARBON MONOXIDE']
x_co = compound_1['x']
y_co = compound_1['y']
compound_3 = shoyu_data_dict['CARBON DIOXIDE']
x_co2 = compound_1['x']
y_co2 = compound_1['y']
peaks_centers1 = spectrafit.compound_report(compound_1)
print(peaks_centers1)
peaks_centers2 = spectrafit.compound_report(compound_2) 
peaks_centers3 = spectrafit.compound_report(compound_3) 
centerlist= [peaks_centers1,peaks_centers2,peaks_centers3]
print(centerlist)

In [ ]:
combined_x12,combined_y12 = shoyu.combine_spectra(compound_1,compound_2, plot = True)

In [ ]:
data_peaks_combine12 = spectrafit.data_report(combined_x12,combined_y12)
peak_1D_score(peaks_centers1,data_peaks_combine12,sorted(set(
    peak_1D_score(peaks_centers1,data_peaks_combine12,1)[0][:]))[-2])

In [ ]:
combined_x13,combined_y13  = shoyu.combine_spectra(compound_1, compound_3, plot = True)

In [ ]:
combined_x23,combined_y23  = shoyu.combine_spectra(compound_2, compound_3, plot = True)

In [ ]:
data_peaks_combine12 = spectrafit.data_report(combined_x12,combined_y12)


In [ ]:
data_peaks_combine13 = spectrafit.data_report(combined_x13,combined_y13)


In [ ]:
data_peaks_combine23 = spectrafit.data_report(combined_x23,combined_y23)


In [ ]:
combinedlist=[data_peaks_combine12,data_peaks_combine13,data_peaks_combine23]

In [ ]:
print(peaks_centers1)
print(data_peaks_combine12)
print(peaks_centers2)
print(combinedlist)
print(centerlist)
compdf = pd.DataFrame(data=score_sort(data_peaks_combine12,peaks_centers1,2)[0][0][:],columns=['WATER_vs_CO_WATER_Scores'])
compdf=compdf.assign(WATER_vs_CO_WATER_Peaks=score_sort(data_peaks_combine12,peaks_centers1,2)[0][1][:])
compdf2=pd.DataFrame(data=score_sort(data_peaks_combine12,peaks_centers2,2)[0][0][:],columns=['CO_vs_CO_WATER_Scores'])
compdf2=compdf2.assign(CO_vs_CO_WATER_scores=score_sort(data_peaks_combine12,peaks_centers2,2)[0][1][:])
# data=score_sort(data_peaks_combine13,peaks_centers1)
# print(data)
print(compdf)
print(compdf2)

In [ ]:
k_range = range(1,len(data_peaks_combine12))
for k in k_range:
    compdf = pd.DataFrame(data=score_sort(data_peaks_combine12,peaks_centers1,k)[0][0][:],columns=['Score for max peak #'+str(k)])
    compdf=compdf.assign(peaks=score_sort(data_peaks_combine12,peaks_centers1,k)[0][1][:])
    print(compdf)

In [ ]:
k_range = range(1,len(data_peaks_combine23))
for i in range(len(combinedlist)):
    for j in range(len(centerlist)):
        for k in k_range:
            compdf = pd.DataFrame(data=score_sort(centerlist[j],combinedlist[i],k)[0][0][:],columns=['Score for max peak k#'+str(k)])
            compdf=compdf.assign(peaks=score_sort(centerlist[j],combinedlist[i],k)[0][1][:])
            print(compdf)

In [ ]:
data=score_sort(centerlist[0],combinedlist[1],1)
print(data)

In [ ]:
scores=score_sort(centerlist[0],combinedlist[0],1)[0][0][:]
peaks=score_sort(centerlist[0],combinedlist[0],1)[0][1][:]
print(peaks)
compdf = pd.DataFrame(data=scores,columns=['WATER_comp_CO_Scores'])
compdf=compdf.assign(WATER__comp_CO_Peaks=peaks)
scores=score_sort(centerlist[0],combinedlist[0],1)[0][0][:]
peaks=score_sort(centerlist[0],combinedlist[0],1)[0][1][:]
print(peaks)
compdf=compdf.assign(Peaks=peaks)
print(compdf)

In [ ]:
compdf = pd.DataFrame(data=scores,columns=['WATER_comp_CO_Scores'])

In [ ]:
# you will need to download the file yourself from the team google drive and edit location
df = pd.read_excel('../examples/FormicAcid_3percentconc_400C_5s_00000.xlsx', names=('x', 'y'))

In [ ]:
fig = plt.figure(figsize=(6,4), dpi = 300)
x_data = df['x'].values
y_data = df['y'].values

In [ ]:
# Exp_peaks = spectrafit.data_report(x_data, y_data)

In [ ]:
compound_1 = shoyu_data_dict['WATER']
compound_2 = shoyu_data_dict['CARBON MONOXIDE']
compound_3 = shoyu_data_dict['ETHYL ALCOHOL']
compound_4 = shoyu_data_dict['FORMIC ACID']
compound_5 = shoyu_data_dict['CARBON DIOXIDE']
H2O_CO_x, H2O_CO_y = shoyu.combine_spectra(compound_1, compound_2, plot = True)
H2O_CO2_x, H2O_CO2_y = shoyu.combine_spectra(compound_1, compound_5, plot = True)

In [ ]:
H2O_CO_x = np.asarray(H2O_CO_x)
H2O_CO_y = np.asarray(H2O_CO_y)
H2O_CO2_x = np.asarray(H2O_CO2_x)
H2O_CO2_y = np.asarray(H2O_CO2_y)

In [ ]:
water_peaks = spectrafit.compound_report(compound_1)[0]
co_peaks = spectrafit.compound_report(compound_2)[0]
co2_peaks = spectrafit.compound_report(compound_5)[0]
H2O_CO_peaks = spectrafit.data_report(H2O_CO_x, H2O_CO_y)
print(H2O_CO_peaks)
H2O_CO2_peaks = spectrafit.data_report(H2O_CO2_x, H2O_CO2_y)


In [ ]:
print(H2O_CO2_peaks)

In [ ]:
Exp_peaks = [355,379,418,587,712,751,814,1034,1219,1272,1383,1400,1640,2138,2943,3185]

In [ ]:
fig = plt.figure(figsize=(6,4), dpi = 300)
plt.plot(x_data, y_data, label ='Experimental')
#plt.plot(H2O_CO2_x, H2O_CO2_y, color = 'red', label = 'Sample Spectra')
plt.xlabel('cm$^{-1}$', fontsize=14)
plt.ylabel('Absoprtion', fontsize=14)
plt.axvline(x = Exp_peaks[9], color = 'green', label='Reference Peak')
plt.axvline(x = Exp_peaks[9]+12, color = 'green', linestyle='--', label='Sample Peak')
plt.axvline(x = Exp_peaks[10], color = 'green')
plt.axvline(x = Exp_peaks[10]+5, color = 'green', linestyle='--')
# plt.axvline(x = H2O_CO2_peaks[5], color = 'red', label='CO2 Reference Peak')
# plt.axvline(x = H2O_CO2_peaks[6], color = 'red')
plt.xlabel('Wavenumber (cm$^{-1}$)', fontsize=12)
plt.ylabel('Counts', fontsize=12)
plt.ylim(0.2,1500)
plt.xlim(1200, 1500)
plt.legend(loc=1, framealpha=1)
#plt.savefig('CO2_Confidence_Interval_plot.png')

In [ ]:
#Ok, now that we have which peaks belong to which component, we'll need to plot their position.
fig = plt.figure(figsize=(10,4), dpi = 300)
plt.plot(x_data, y_data, color = 'black', label = 'Sample Spectra')
plt.axvline(x = Exp_peaks[0], color = 'black', label = 'Hydrogen',linestyle='--')
#plt.axvline(x = Exp_peaks[1], color = 'yellow',alpha=.7)
plt.axvline(x = Exp_peaks[2], color = 'yellow', label = 'Sapphire',alpha=.7)
plt.axvline(x = Exp_peaks[3], color = 'black', linestyle='--')
plt.axvline(x = Exp_peaks[4], color = 'black', linestyle='--')
plt.axvline(x = Exp_peaks[5], color = 'green', label = 'Formic Acid')
plt.axvline(x = Exp_peaks[5]+50, color = 'green')
plt.axvline(x = Exp_peaks[6], color = 'black', linestyle='--')
plt.axvline(x = Exp_peaks[7], color = 'black', linestyle='--')
#plt.axvline(x = Exp_peaks[8], color = 'green')
plt.axvline(x = Exp_peaks[9], color = 'red', label = 'Carbon Dioxide')
plt.axvline(x = Exp_peaks[10], color = 'red')
plt.axvline(x = Exp_peaks[11], color = 'green')
plt.axvline(x = Exp_peaks[12], color = 'blue',label = 'Water Match')
plt.axvline(x = Exp_peaks[13], color = 'purple', label = 'Carbon Monoxide')
#plt.axvline(x = Exp_peaks[14], color = 'green')
plt.axvline(x = Exp_peaks[15], color = 'blue')
plt.legend(loc=1, framealpha=1)
plt.xlabel('Wavenumber (cm$^{-1}$)', fontsize=12)
plt.ylabel('Counts', fontsize=12)
#plt.ylim(-0.1, 1.3)
# plt.ylim()
    
#     plt.axvline(x=H2O_CO_x[i], color='orange')
#plt.savefig('Experimental_In_situ_Reference.png')